<a href="https://colab.research.google.com/github/Mikhail-Klochkov/ml_intro/blob/master/Recsystem_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание по рекомендательным системам

В данном домашнем задании вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

При запуске тестов могут появиться предупреждения: PearsonRConstantInputWarning и PearsonRNearConstantInputWarning. На них можно не обращать внимания.

Возможный максимум баллов за задание: 10 баллов <br>
Дедлайн: ??? <br>
Штраф: ??? - будет ли в курсе штраф? <br>
<br>
Для ускорения проверки, напишите здесь получившееся количество баллов: (Не знаю, но тесты все прошёл файлик utils.py and test.py).

## 1. Метрика сходства
<b>1.1. Реализация метрик (2 балла)</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обоих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

<b>1.2. (1 балл)</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$, где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [191]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [192]:
import numpy as np
import drive.MyDrive.Colab_Notebooks.recomendation_system.utils as utils
from drive.MyDrive.Colab_Notebooks.recomendation_system.utils import euclidean_similarity, euclidean_distance, pearson_similarity


def disteuclid(x, y):
  return np.sqrt(((x - y) ** 2).sum())

def simeuclid(x, y):
  return 1 / ( 1 + disteuclid(x, y))

def euclidean_similarity2_colab(x: np.ndarray, y: np.ndarray) -> float:
    mask = ((x != 0) & (y != 0))
    x_not_zero = x[mask]
    y_not_zero = y[mask]
    # -- not exist intersection -- #
    return simeuclid(x_not_zero, y_not_zero) if x_not_zero.shape[0] != 0 else -np.inf

def pearson_similarity2_colab(x: np.ndarray, y : np.ndarray ) -> float:
    mask = ((x != 0) & (y != 0))
    x_not_zero = x[mask]
    y_not_zero = y[mask]
    # -- not exist intersection -- #
    return pearson_similarity(x_not_zero, y_not_zero) if x_not_zero.shape[0] != 0 else -np.inf

def PS_colab(x: np.ndarray, y : np.ndarray ) -> float:
    mask = ((x != 0) & (y != 0))
    x_not_zero = x[mask]
    y_not_zero = y[mask]
    # -- not exist intersection -- #
    dx = x_not_zero - x.mean()
    dy = y_not_zero - y.mean()
    return np.dot(dx, dy)/(np.linalg.norm(dx) * np.linalg.norm(dy)) if x_not_zero.shape[0] > 1 else -np.inf

def PSweighted_colab(x: np.ndarray, y : np.ndarray ) -> float:
  maskint = ((x != 0) & (y != 0))
  maskun = ((x != 0) | (y != 0))
  x_not_zero = x[maskint]
  y_not_zero = y[maskint]
  dx = x_not_zero - x.mean()
  dy = y_not_zero - y.mean()
  weight = maskint.sum()/maskun.sum()
  return weight * np.dot(dx, dy)/(np.linalg.norm(dx) * np.linalg.norm(dy)) if x_not_zero.shape[0] > 1 else -np.inf
  
def Eweighted_colab(x: np.ndarray, y: np.ndarray) -> float:
    maskint = ((x != 0) & (y != 0))
    maskun = ((x != 0) | (y != 0))
    x_not_zero = x[maskint]
    y_not_zero = y[maskint]
    # -- not exist intersection -- #
    weight = maskint.sum()/maskun.sum()
    return weight * euclidean_similarity(x_not_zero, y_not_zero) if len(x_not_zero) != 0 else -np.inf 



In [193]:
! wget "https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv" 


--2021-05-12 22:50:32--  https://www.kaggle.com/rounakbanik/the-movies-dataset
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘the-movies-dataset.1’

the-movies-dataset.     [  <=>               ]  62.91K   152KB/s    in 0.4s    

2021-05-12 22:50:34 (152 KB/s) - ‘the-movies-dataset.1’ saved [64416]



## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики может для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [402]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter

class UserBasedRecommendation:
    def __init__(self, 
                 metric: str = 'euclidean', 
                 n_recommendations: int = 5, 
                 alpha: float = 0.8,
                 verbose = 0):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        self._metric = metric
        self._n_recomendations = n_recommendations
        self._alpha = alpha 
        self._verbose = verbose

    def fit(self, X):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        # -- sparse matrix -- #
        if(not isinstance(X, np.ndarray)):
          self.X_ = np.copy(X.todense())
        else:
          self._X = np.copy(X)

        print("shape X matrix: {}".format(self._X.shape))

    def find_closest_users(self, 
                             user_id: int, 
                             n_closest_users: int):
      
        """
        user_id - it's index in array self._X
        n_closest_users - it's number of closest users to user_id
        return: dists and closest indeces users
        """
        # -- Что подавать оригинальный или нет user_id -- #
        # -- Количество n_closest_users -- #
        # -- d = 1 - sim(u, v) -- #
        # -- по идее в группу схожих объектов надо убрать его самого, а то мы можем таким образом посоветовать, что уже пользователь смотрел -- #
        
        assert(isinstance(self._X, np.ndarray))
        x0 = self._X[user_id].reshape(1, self._X.shape[1])
        assert(x0.shape.__len__() == 2)
        # -- + 1 сделать поправку на пользователя, которые представляет самого себя в массиве данных -- #
        NB = NearestNeighbors(n_neighbors = n_closest_users + 1, 
                         metric = sim2metrics(Eweighted) 
                                  if self._metric == 'euclidean' else 
                                  sim2metrics(PSweighted))
        NB.fit(self._X)
        # -- get distances metric(x1, x2) and indeces closest users with x0 element -- #        
        dists, indeces = NB.kneighbors(x0)
        # -- [1:] - не выводит самого себя среди списка -- #
        return dists[0][1: ], indeces[0][1:]

# -- Optional need for x : int  = None without errors -- #
    def make_recommendation(self, 
                            user_id: int, 
                            n_recommendations: Optional[int] = None):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        # -- Немного не ясно зачем f_closest принимает количество ближайших, если у нас есть параметр alpha, который позволяет -- # 
        # -- Определить окрестность (соседей), которые ближайшие к данному user_id -- #
        dists, indeces = self.find_closest_users(user_id, self._X.shape[0]-1)
        # -- sim(p, q) = 1 - d(p, q)
        neighbors_idxs = indeces[((1 - dists) > self._alpha)]
        if(self._verbose > 0):
          print('userID: {} number of neighbors: {}'.format(user_id, ((1 - dists) > self._alpha).sum()))

        # -- Из этого же множества объектов для каждого фильма, должны посмотреть p_i -- #
        # -- Все при условии, что матрица будет типа np.ndarray -- #
        P = np.array([
              (self._X[neighbors_idxs, item_idx] != 0).sum()/len(neighbors_idxs)
              for item_idx in range(self._X.shape[1])
            ])
        if(self._verbose > 0):
          print('p sizes: {}'.format(P.shape[0]))
        # -- sort and get top n_recomendations -- #
        # -- Зачем нам в параметрах класса иметь n_recomendations и передавать данный аргумент в функцию аттрибута класса, они что, разные? -- #
        topidxitems = P.argsort()[::-1][:n_recommendations]
        # -- Вернутся преобразованные индексы -- #
        # -- Потом словарём надо их переименовать обратно -- #
        return topidxitems
        

        

Ниже я делаю некоторую обёртку, которая по переданной функции similarities возвращает метрику $1-sim(p,q) = d(p, q)$. А также обычная функция, корая по переданному корпусу данных, одного объекта и переданной метрики, возвращает к ближайших соседей.

In [403]:
def sim2metrics(sim):
  """
  wrapper
  x, y - np.ndarray 1-d
  get similarity function sim(x, y)
  """
  def metric(x, y):
    """
    x, y - is np.ndarray
    """
    return 1 - sim(x, y)
  return metric

def getkNearestNeighbors(X: np.ndarray, 
                         x: np.ndarray,
                         metric,  
                         k : int = 5,
                         metric_params = {}):
  """
  get X array and x 1-d vector
  metric - is callable function 2 arguments
  metric params for metric function
  """
  distances = np.array([metric(x, X[idx]) for idx in range(X.shape[0])])
  indeces = distances.argsort()[: k]
  return distances[indeces], indeces

## 4. Применение модели
<b>4.1. (2 балла)</b>

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [505]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [506]:
import pandas as pd
file_path = "/content/drive/MyDrive/Colab_Notebooks/recomendation_system/ratings_small.csv"
df_ratings = pd.read_csv(file_path)

In [507]:
print("shape of data: {}".format(df_ratings.shape))
shape_begin = df_ratings.shape

shape of data: (100004, 4)


In [508]:
df_ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [509]:
df_ratings.rating.value_counts()

4.0    28750
3.0    20064
5.0    15095
3.5    10538
4.5     7723
2.0     7271
2.5     4449
1.0     3326
1.5     1687
0.5     1101
Name: rating, dtype: int64

In [510]:
df_ratings.userId.min(), df_ratings.userId.max(), len(df_ratings.userId.unique())

(1, 671, 671)

In [511]:
df_ratings.movieId.min(), df_ratings.movieId.max(), len(df_ratings.movieId.unique())

(1, 163949, 9066)

Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

Удалим для наиболее активных пользователей 50 оценок. Для очень популярных пользователей брать 5 фильмов всего для  выбрасывания - маловато. Будет очень низкая оценка. 

In [512]:
num_films = 30
active_users = df_ratings.userId.value_counts()[:10].index
# -- Мы не убирали пока часть данных -- #
if(df_ratings.shape[0] == shape_begin[0]):
  test_data = pd.DataFrame([], columns=df_ratings.columns)
  for user_id in active_users:
      _, test = train_test_split(df_ratings[df_ratings.userId == user_id], 
                                test_size = num_films, 
                                random_state = 123)
      test_data = test_data.append(test, ignore_index = True)
      mask = ((df_ratings.userId == user_id) & (df_ratings.movieId.isin(test.movieId.values)))
      # -- Убираем часть данных -- #
      df_ratings = df_ratings[~mask]

df_ratings.shape, test_data.shape

((99704, 4), (300, 4))

In [517]:
# -- by userId get continious id -- #
userid_2_contuserid = {
    usid : id 
    for id, usid in enumerate(df_ratings.userId.unique(), start = 0)
    }
# -- by continious id get userId -- #
contuserid_2_userid = {idx: usid 
                       for usid, idx in userid_2_contuserid.items()
                       }
# -- by  movieid get continious id -- #                       
movieid_2_contuserid = {
    mid : id 
    for id, mid in enumerate(df_ratings.movieId.unique(), start = 0)
    }
# -- by continious id get movieid -- #                       

contuserid_2_movieid = {idx: mid 
                       for mid, idx in movieid_2_contuserid.items()
                       }                    

# -- rename user ids -- #
df_ratings.userId = df_ratings.userId.apply(lambda userid: userid_2_contuserid[userid])
# -- rename movies ids -- #
df_ratings.movieId = df_ratings.movieId.apply(lambda mid: movieid_2_contuserid[mid])

# -- test data also rename -- #
test_data.userId = test_data.userId.apply(lambda userid: userid_2_contuserid[userid])

test_data.movieId = test_data.movieId.apply(lambda mid: movieid_2_contuserid.get(mid, -1))

In [518]:
df_ratings.userId.min(), df_ratings.userId.max(), len(df_ratings.userId.unique())

(0, 670, 671)

In [519]:
df_ratings.movieId.min(), df_ratings.movieId.max(), len(df_ratings.movieId.unique())

(0, 9038, 9039)

Хотелось бы для каждого пользователя посмотреть количество общих фильмов со всеми другими пользователями. Или отношение количества общих фильмов для каждой пары пользователей p, q к количеству объединенных фильмов пары p,q - |I(p) /\ I(q)| / |I(p) \/ I(q)|. Такое отношение посмотреть. Это может в некотором примитивном приближении показать насколько полтзователи в целом смотрят одинаковые фильмы именно в этом данном dataset.  

**Ниже просто экспериментирую с самым первым что приходит на ум. Чтобы было что сравнить с предложенным подходом в классе UserBased system.** Суть в том, что давайте выкинем из данных для каждого пользователя по 50 филмов, чтобы вероятность выдать просмотренный, но удалённый фильм из данных был выше (всего уникальных фильмов ~10 000). Потом будем считать, сходство пользователей исключительно по sim(p, q) = |I(p) /\ I(q)| / |I(p) / I(q)|. И по этому показателю выделять ближайших объектов.И рекомендовать фильмы по той же схеме, как и в UserBasedSystem. Как окажется, что алгоритм вообще говоря редко будет выдавать какие-то фильмы, которые мы выкинули выше! Поэтому данный подход достаточно плохой, хуже чем UserBasedSystem. Конечно, там ещё можно по подбирать оптимальным образом количество схожих пользователей, но лучшге от этого не особо станет. 

In [520]:
# -- mask -- #

matrix_masked = np.zeros((df_ratings.userId.unique().shape[0], 
                          df_ratings.movieId.unique().shape[0]), 
                          dtype = np.int16)

unique_users = df_ratings.userId.unique()
print("matrix_masked shape: {}".format(matrix_masked.shape))
group_users = df_ratings.groupby(by = 'userId')
for uidx in unique_users:
  df_uidx = group_users.get_group(uidx)
  mask_uidxfilms = np.zeros((matrix_masked.shape[1], ), np.int16)
  mask_uidxfilms[df_uidx['movieId'].values] = 1
  matrix_masked[uidx, :] = mask_uidxfilms

# -- created -- #
common_films = matrix_masked @ matrix_masked.T
assert(common_films.shape == (unique_users.shape[0], 
                              unique_users.shape[0]))
common_films

matrix_masked shape: (671, 9039)


array([[ 20,   0,   0, ...,   1,   0,   1],
       [  0,  76,   8, ...,   2,   8,   9],
       [  0,   8,  51, ...,   3,   5,  11],
       ...,
       [  1,   2,   3, ...,  37,   1,   4],
       [  0,   8,   5, ...,   1,  31,  13],
       [  1,   9,  11, ...,   4,  13, 115]], dtype=int16)

In [521]:
import numpy.ma as ma
common_films_pd = pd.DataFrame([], 
                               columns = ['user1', 
                                          'user2', 
                                          'num_common_films'])

# -- very slow -- #
"""
for idx in range(common_films.shape[0] - 1):
  for jdx in range(idx + 1, common_films.shape[0], 1):  
    dict_ = {'user1': idx, 
             'user2': jdx, 
             'num_common_films' : common_films[idx, jdx]}
    common_films_pd = common_films_pd.append(dict_, 
                                             ignore_index=True)
"""    
mask = np.full((common_films.shape[0], common_films.shape[0]), 
               fill_value = 1, 
               dtype = np.uint8)

maxes = []
for uidx in range(common_films.shape[0] - 1):
  mask[uidx, np.arange(uidx + 1, common_films.shape[0], 1)] = 0 
  maxes.append(common_films[uidx, uidx+1: ].max())

# -- Думал пригодиться -- #
common_films_masked = ma.array(common_films, 
                               mask = mask)
maxes = np.array(maxes)
# -- Для каждого фильма есть много пересечений, то есть есть пользователи, которые смотрят достаточно много общего -- #
# -- Чтобы подсчитать для каждой пары  |I(p) /\ I(q)| / |I(p) / I(q)| создадим матрицу с знаменателями -- #
# -- Хорошая функция apply_along_axis -- #

times = -1
def funct1(row, diag):
  global times
  times += 1
  return row/(diag[times] + common_films.diagonal())

num_users = common_films.shape[0]
# -- Матрица |I(p) /\ I(q)| / |I(p) / I(q)| значений размером num_users*num_users -- #
weights_intersection = np.apply_along_axis(funct1, 
                                           1,
                                           common_films, 
                                           common_films.diagonal())
weights_intersection

array([[0.5       , 0.        , 0.        , ..., 0.01754386, 0.        ,
        0.00740741],
       [0.        , 0.5       , 0.06299213, ..., 0.01769912, 0.07476636,
        0.04712042],
       [0.        , 0.06299213, 0.5       , ..., 0.03409091, 0.06097561,
        0.06626506],
       ...,
       [0.01754386, 0.01769912, 0.03409091, ..., 0.5       , 0.01470588,
        0.02631579],
       [0.        , 0.07476636, 0.06097561, ..., 0.01470588, 0.5       ,
        0.0890411 ],
       [0.00740741, 0.04712042, 0.06626506, ..., 0.02631579, 0.0890411 ,
        0.5       ]])

In [522]:
# -- 0.5 значение говорит о том, что у пары пользователей есть 50 % общих просмотренных фильмов -- #

# -- global variable weights_intersection -- #
def get_knearest_users(user_id, k : int = 5):
  """
  По построенной матрице, мы определяем с точки зрения данных 
  значений ближайших по общему пересечению объектов 
  returned - топ без самого себя конечно.
  """
  return weights_intersection[user_id, :].argsort()[::-1][1: k + 1]

def make_recomendations(df_train,
                        user_id, 
                        top_films: int = 150, 
                        knearest : int = 50,
                        import_rating = False):
  # -- выделить похожих пользователей => (пользователи с большим относительным пересечением в просмотре фильмов) -- #
  groupusers = get_knearest_users(user_id, knearest)
  #group_users_df = df_train.groupby(by = ['userId'])
  # -- see on each film and calculate pi -- #
  maskusers = df_train.userId.apply(lambda userid: userid in set(groupusers))
  P = np.array([
        len(df_train[(df_train.movieId == filmid) & maskusers].userId.unique()) / len(groupusers)
        for filmid in df_train.movieId.unique() 
      ])
  filmindeces = P.argsort()[::-1][: top_films]
  return P[filmindeces], filmindeces


user_id_test = test_data.sample().userId.values[0]
print('userid: ', user_id_test)
Ps, topfilms = make_recomendations(df_ratings, 
                    user_id_test)
np.sort(topfilms)

userid:  546


array([  19,   23,   24,   27,   29,   38,   40,   42,   49,   57,   58,
         59,   64,   69,   72,   75,   79,   86,   87,   88,   89,   90,
         91,   92,   99,  101,  102,  105,  106,  110,  111,  113,  118,
        119,  121,  122,  139,  143,  146,  153,  157,  159,  161,  170,
        171,  172,  173,  174,  177,  179,  180,  181,  182,  183,  184,
        185,  187,  188,  189,  190,  191,  194,  195,  196,  197,  199,
        201,  202,  204,  213,  214,  225,  227,  229,  256,  263,  264,
        272,  278,  287,  289,  290,  297,  298,  300,  324,  326,  327,
        328,  329,  330,  332,  334,  335,  339,  341,  343,  358,  385,
        389,  402,  403,  412,  417,  418,  432,  434,  435,  441,  445,
        447,  457,  459,  468,  470,  480,  483,  505,  517,  519,  527,
        551,  584,  690,  714,  720,  732,  733,  736,  738,  742,  767,
        774,  776,  780,  785,  796,  801,  814,  824,  825,  874,  875,
       1001, 1024, 1027, 1033, 1037, 1116, 2150])

In [523]:
Ps

array([0.96, 0.92, 0.92, 0.9 , 0.9 , 0.88, 0.88, 0.88, 0.86, 0.86, 0.84,
       0.84, 0.84, 0.84, 0.84, 0.82, 0.82, 0.82, 0.82, 0.8 , 0.8 , 0.8 ,
       0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.78, 0.78, 0.78, 0.78, 0.76, 0.76,
       0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.74, 0.74, 0.74, 0.72, 0.72,
       0.72, 0.72, 0.72, 0.72, 0.72, 0.72, 0.72, 0.7 , 0.7 , 0.7 , 0.7 ,
       0.7 , 0.7 , 0.7 , 0.7 , 0.7 , 0.7 , 0.68, 0.68, 0.68, 0.68, 0.68,
       0.68, 0.68, 0.68, 0.68, 0.68, 0.66, 0.66, 0.66, 0.66, 0.66, 0.66,
       0.66, 0.66, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64,
       0.64, 0.64, 0.64, 0.64, 0.62, 0.62, 0.62, 0.62, 0.62, 0.62, 0.62,
       0.62, 0.62, 0.62, 0.62, 0.62, 0.62, 0.62, 0.6 , 0.6 , 0.6 , 0.6 ,
       0.6 , 0.6 , 0.6 , 0.6 , 0.6 , 0.58, 0.58, 0.58, 0.58, 0.58, 0.58,
       0.58, 0.58, 0.58, 0.58, 0.58, 0.58, 0.58, 0.58, 0.58, 0.58, 0.58,
       0.58, 0.58, 0.58, 0.58, 0.58, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56,
       0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56])

In [524]:
deletedmovies = test_data.groupby(by = 'userId').get_group(user_id_test).movieId
deletedmovies

0     5963
1     4564
2       -1
3     5034
4     3426
5       -1
6     4123
7     8331
8     5238
9     8667
10    1307
11      -1
12    3706
13    4226
14    6852
15    6711
16    4992
17    1198
18      -1
19      -1
20     628
21    8507
22    6021
23    3096
24    2017
25    3833
26      -1
27    1268
28    1960
29     176
Name: movieId, dtype: int64

**Пересечение в примитивном подходе**

In [525]:
print("пересечение между рекомендованными и теми, что были удалены у пользователя: ", np.intersect1d(topfilms, deletedmovies))

пересечение между рекомендованными и теми, что были удалены у пользователя:  []


Можно видеть достаточно малое пересечение! Оно и ясно, вообще говоря факт того, что пользователь посмотрел фильм не гарантирует его актуальность. 

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [526]:
import scipy.sparse as scp
def get_sparse(data,
               colr :str,
               colu :str,
               colf :str
               ):
# -- colr, colu, colf -- #
    return scp.coo_matrix(
        (
            data[colr],  
            (data[colu], data[colf])
        ), 
        shape=(len(userid_2_contuserid), len(movieid_2_contuserid))  # размеры матрицы рейтингов
    ).tocsr()

X_sp = get_sparse(df_ratings,
                  colr = 'rating',
                  colu = 'userId',
                  colf = 'movieId')
X_sp

<671x9039 sparse matrix of type '<class 'numpy.float64'>'
	with 99704 stored elements in Compressed Sparse Row format>

Все численные эксперименты проводить ниже.

In [527]:
from drive.MyDrive.Colab_Notebooks.recomendation_system.utils import euclidean_similarity2, Eweighted, PSweighted, apk, mapk

In [528]:
X_ex = np.copy(X_sp[1:].todense())
x0 = np.copy(X_sp[0].todense()).reshape((X_sp.shape[1], ))

metric_use = sim2metrics(Eweighted)
dists, indeces = getkNearestNeighbors(X_ex, 
                                      x0, 
                                      metric = metric_use
                                     )  

dists, indeces

(array([0.95286195, 0.96035676, 0.96340362, 0.96949904, 0.97515074]),
 array([323, 308, 632,  33, 602]))

In [529]:

X_ex = np.copy(X_sp[1:].todense())
x0 = np.copy(X_sp[0].todense()).reshape((X_sp.shape[1], ))

metric_use = sim2metrics(PSweighted)
dists, indeces = getkNearestNeighbors(X_ex, 
                                      x0, 
                                      metric = metric_use
                                     )  

dists, indeces.__len__()

(array([0.80878858, 0.86905229, 0.8941205 , 0.92003451, 0.92134547]), 5)

In [530]:
class_ = UserBasedRecommendation(alpha = 0.2)
class_.fit(X_sp.todense())

dists, indeces = class_.find_closest_users(1, n_closest_users = 10)
dists.shape, indeces.shape
1-dists

shape X matrix: (671, 9039)


array([0.05714933, 0.0558315 , 0.0546354 , 0.0543141 , 0.04908214,
       0.04879487, 0.04792717, 0.04714778, 0.04657866, 0.04451107])

<b>2.2. (1 балла)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

In [460]:
# -- Вообще говоря мне в целом странным кажется использовать данные меры сходства, особенно корреляцию пирсона -- #
pass

<b>Объяснение:</b> Для ясности пронумеруем товары с 0. Видно, что третьему пользователю так же как и второму больше понравился товар 1 и чуть меньше понравился товар 2, тогда как первому пользователю оба эти товары не сильно понравились, причем товар 1 ему понравился меньше, чем товар 2, то есть наблюдается обратная завизимость. Но все эти факторы никак не учитываются при использовании евклидовой метрики, поэтому в первом случае алгоритм посчитал, что первый и третий пользователь похожи только по тому, что они ставят оценки примерно в одном и том отрезке: {0, 1}, что является довольно странным предположением. А алгоритм, использующий коэффициент корреляции Пирсона учитывает эти факторы, поэтому находит третьему пользователю соседа со схожими интересами в виде второго пользователя. Отсюда получаем разные рекомендации.

## 3. Оценка качества
<b>3.1. (1 балл)</b>

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

Ниже по UserBasedRecommendation - нахожу топ 100 фильмов и ниже пытаюсь оценить (считая в test_data, что фильм релевантный, если его оценка выше чем медианное значения для каждого отдельно взятого пользователя).

In [531]:
import warnings 
warnings.filterwarnings('ignore')
# -- Что значит подобрать alpha? (по кроссвалидации как гиперпарамметр) -- #
recomendclass = UserBasedRecommendation(metric = 'pearson',
                                        n_recommendations = 100, 
                                        alpha = 0.095,
                                        verbose = 1)

# -- fit -- #
recomendclass.fit(X_sp.todense())
# -- Для наиболее активных пользователей топ 10 мы убрали по 5 случайных мувиков, которые он посмотрел -- #
group_test = test_data.groupby(by = 'userId')
topfilms = {}
user_2_num_relevants_test = {}
user_2_relevant_films = {}
user_2_non_relevant_films = {}

for it, us_idx in enumerate(test_data.userId.unique(), start = 1):
  filmidxs = recomendclass.make_recommendation(us_idx,
                                               n_recommendations = 100)
  topfilms[us_idx] = filmidxs
  assert(filmidxs.shape[0] == np.unique(filmidxs).shape[0])
  print('userId: {} num of ratings: {}'.format(us_idx, 
                                               df_ratings[df_ratings.userId == us_idx].shape[0]))
  # -- количество оценок в тестовой части данных -- #
  #stars_idx = df_R.loc[us_idx, (df_R.loc[us_idx, :].notna())]
  # -- Мы можем подсчитать для каждого пользователя median оценку и считать всё выше релевантным все ниже нерелевантным -- #
  median_idx = df_ratings[df_ratings.userId == us_idx].rating.median()
  print('idx: {} median: {}'.format(us_idx, median_idx))
  # -- Воспользуемся рассчитанной медианной для расчёта количества релевантных пользователей -- #
  user_2_num_relevants_test[us_idx] = (group_test.get_group(us_idx).rating >= median_idx).sum()
  # -- calculate relevant users -- #
  df_user_test = group_test.get_group(us_idx)
  user_2_relevant_films[us_idx] = df_user_test[df_user_test.rating >= median_idx].movieId.values
  # -- calculate non relevant users -- #
  user_2_non_relevant_films[us_idx] = df_user_test[~(df_user_test.rating >= median_idx)].movieId.values



shape X matrix: (671, 9039)
userID: 546 number of neighbors: 32
p sizes: 9039
userId: 546 num of ratings: 2361
idx: 546 median: 3.5
userID: 563 number of neighbors: 49
p sizes: 9039
userId: 563 num of ratings: 1838
idx: 563 median: 4.0
userID: 623 number of neighbors: 50
p sizes: 9039
userId: 623 num of ratings: 1705
idx: 623 median: 3.0
userID: 14 number of neighbors: 82
p sizes: 9039
userId: 14 num of ratings: 1670
idx: 14 median: 3.0
userID: 72 number of neighbors: 84
p sizes: 9039
userId: 72 num of ratings: 1580
idx: 72 median: 3.5
userID: 451 number of neighbors: 74
p sizes: 9039
userId: 451 num of ratings: 1310
idx: 451 median: 3.0
userID: 467 number of neighbors: 94
p sizes: 9039
userId: 467 num of ratings: 1261
idx: 467 median: 3.0
userID: 379 number of neighbors: 84
p sizes: 9039
userId: 379 num of ratings: 1033
idx: 379 median: 3.5
userID: 310 number of neighbors: 76
p sizes: 9039
userId: 310 num of ratings: 989
idx: 310 median: 3.0
userID: 29 number of neighbors: 88
p sizes:

**Ниже словарь - каждому уникальному индексу пользователя ставиться в соответствие запись: (число релевантных фильмов, которые нашёл алгоритм/ числу релевантных алгоритмов в test_data)** Видно, что результаты выглядят печальненько... С другой стороны, ведь фильмов ~ 10000, может быть эти релевантные для себя фильмы пользователь ещё не посмотрел, и само выделение test_data происходит абсолютно случайно!!!

In [532]:
""" # -- У нас есть среди неизвестных оценок выбранных пользователей некоторые оценки,
    которые мы разметили некоторым примитивным образом. В первом приближении лучше посмотреть будут ли
    рекомендованные фильмы, которые были получены с помощью нашего алгоритма содержать данные фильмы! -- """

# -- Формально именно здесь мы смотрим на те ответы, которые есть в тесте, а не в обучающей выборке -- # 
user_2_top = {}
user_2_num_in_top = {}
sum_in_times = 0
for uidx in test_data.userId.unique():
  # -- uidx_d {index of film : position in top} -- #
  uidx_d = {}
  times_in = 0
  for fidx in user_2_relevant_films[uidx]:
    # -- либо найтись единственный / либо не найтись -- #
    args = np.argwhere(topfilms[uidx] == fidx) 
    uidx_d[fidx] = -1 if( args.shape[0] == 0 ) else args[0][0] 
    times_in += 1 if uidx_d[fidx] != -1 else 0
  user_2_top[uidx] = uidx_d
  sum_in_times += times_in
  user_2_num_in_top[uidx] = str(times_in) + '/' + str(user_2_relevant_films[uidx].shape[0])

user_2_num_in_top, sum_in_times

({14: '0/10',
  29: '1/23',
  72: '0/21',
  310: '0/17',
  379: '0/20',
  451: '2/23',
  467: '0/17',
  546: '0/18',
  563: '1/17',
  623: '0/18'},
 4)

In [533]:
"""
 Стоит попробовать поперебирать коэффициент alpha 
 чтобы лучше получать результаты, хотя бы чтобы ка можно меньше было -1 - 
 которые говорят о том, что среди предсказанного топа элементов нет релевантных фильмов в том смысле, 
 что оценка релевантного фильма для каждого пользователя определяется из соображений, что она больше 
 медианного значения для фиксированного пользователя. 

 Конечно это спорный момент, но мы хотим получить хоть что-то разумное в первом приближении прежде чем
 говорить о каких-то содержательных метриках качества apk and mapk, которые помимо всего этого 
 учитывают порядок в котором идут топ эелементов! 
"""

'\n Стоит попробовать поперебирать коэффициент alpha \n чтобы лучше получать результаты, хотя бы чтобы ка можно меньше было -1 - \n которые говорят о том, что среди предсказанного топа элементов нет релевантных фильмов в том смысле, \n что оценка релевантного фильма для каждого пользователя определяется из соображений, что она больше \n медианного значения для фиксированного пользователя. \n\n Конечно это спорный момент, но мы хотим получить хоть что-то разумное в первом приближении прежде чем\n говорить о каких-то содержательных метриках качества apk and mapk, которые помимо всего этого \n учитывают порядок в котором идут топ эелементов! \n'

По формуле: (Которая есть аналог ядерной регресии, где sim() - возьмём как cosine)

$$
  R^{'}_{i} = \overline R_{a} + \frac{\sum_{j \in KNN(i)} sum(i, j)*(R_{j, a} - \overline R_{j} ))}{\sum_{j \in KNN(i)} sim(i, j)}
$$

In [564]:
# -- Другие методы -- #

def cosine_similarities(x, y):
  """
  """
  return np.dot(x, y)/(np.linalg.norm(x) * np.linalg.norm(y))

# -- User_Items -- #
X_dense = np.copy(X_sp.todense())
# -- masked -- #
mask = (X_dense == 0)
X_masked = ma.array(X_dense, mask = mask)
films_mean = X_masked.mean(axis = 0)
users_mean = X_masked.mean(axis = 1)
print(users_mean.shape, films_mean.shape)
# -- сделаем предсказания только для тех данных, что мы выкинули -- #
# -- Для разных функций sim() -- #
preds = []
for row in test_data.itertuples():
  userid = getattr(row, 'userId')
  filmid = getattr(row, 'movieId')
  # -- calculate -- #
  metric_ = sim2metrics(cosine_similarities)
  # -- в сумме будет участвовать только ближайшие -- #
  dists, indeces = getkNearestNeighbors(X_dense, 
                       X_dense[userid, :],
                       metric = metric_, 
                       k = 50)
  # -- dist(p, q) = 1 - sim(p, q) -- #
  try:
    sum_ = sum([
            (X_dense[uidx, filmid] - users_mean[uidx]) * (1 - dists[idx])/sum(np.fabs(dists))
            for idx, uidx in enumerate(indeces)
           ])
  except IndexError:
    print(filmid, userid)
  # -- write ans -- #
  r = users_mean[userid] + sum_
  preds.append((userid, filmid, r))

preds = np.asarray(preds)
# -- Обрезать что больше и меньше [0, 5]-- #
preds[np.array([preds[idx][2] for idx in range(len(preds))]) < 0] = 0.5
preds[np.array([preds[idx][2] for idx in range(len(preds))]) > 5.0] = 5.



(671,) (9039,)


RMSE ответы на ratings на тестовых данных (можно поварьрировать количество ближайших):

In [565]:
from sklearn.metrics import mean_squared_error

preds_rating = preds[:, 2]
print("rmse: {:.3f}".format(np.sqrt(mean_squared_error(test_data.rating.values, preds_rating)/preds.shape[0])))

rmse: 0.118


Чтобы подсчитать какие-нибудь метрики аналогичные APK, MAPK стоит определить, что называть релевантным фильмов и не релевантным. Выборка данных не позволяет точно ответить на данный вопрос. Я решил просто считать релевантым объект у которого оценка выше медианной для каждого объекта.

In [593]:
# -- Попробуем с помощью данного метода определять релевантные -- #

metric_ = sim2metrics(cosine_similarities)
ans_all = {} 
for uidx in test_data.userId.unique()[:]:
  print(uidx)
  metric_ = sim2metrics(cosine_similarities)
  dists, indeces = getkNearestNeighbors(X_dense, 
                                                X_dense[uidx, :],
                                                metric = metric_, 
                                                k = 11)
  # -- подсчитать ответы только на тех индексах фильма, которых ещё не видел текущий пользователь -- #
  # -- 0.0 которые ещё не видел -- #
  dists = dists[1:]
  indeces = indeces[1:]

  non_ans_films = np.arange(X_dense.shape[1])[(X_dense[uidx, :] == 0)]
  print('for user: {} number not seen films : {}'.format(uidx, len(non_ans_films)))
  r_dict = {}
  for filmid in non_ans_films:
    try:
      sum_ = sum([
              (X_dense[uid, filmid] - users_mean[uid]) * (1 - dists[idx])/sum(np.fabs(dists))
              for idx, uid in enumerate(indeces)
            ])
    except IndexError:
      print(filmid, uidx)

    r = users_mean[userid] + sum_
    # -- обрежем, если нужно -- #
    r_dict[filmid] = np.clip(0, 5, r)  
  # -- для каждого фильма выделяем top N -- #
  N = 100
  topfilms = np.array([ filmidx[0]
             for filmidx in sorted(r_dict.items(), key = lambda x: x[1])[-N:]])

  topfilms = topfilms[::-1]
  ans_all[uidx] = topfilms

546
for user: 546 number not seen films : 6678
563
for user: 563 number not seen films : 7201
623
for user: 623 number not seen films : 7334
14
for user: 14 number not seen films : 7369
72
for user: 72 number not seen films : 7459
451
for user: 451 number not seen films : 7729
467
for user: 467 number not seen films : 7778
379
for user: 379 number not seen films : 8006
310
for user: 310 number not seen films : 8050
29
for user: 29 number not seen films : 8058


In [599]:
# -- Для каждого фильма можно подсчитать значения apk and another -- #
apkall = {}
K = [5, 10, 100]
TD = test_data.groupby(by = 'userId')
for uidx, preds in ans_all.items():
  # test_data we have unknown #
  dfidx = TD.get_group(uidx)
  medianidx = dfidx.rating.median() 
  relevantmovies = dfidx[(dfidx.rating > median_idx)].movieId
  print("for userid: {} number of relevant: {} and median rating: {}".format(uidx, 
                                                       relevantmovies.shape[0],
                                                       medianidx))
  # -- apk for each -- #
  apklist = []
  for k in K:
    apk_k = apk(relevantmovies, preds, k = k)
    apklist.append(apk_k)

  apkall[uidx] = apklist

for userid: 546 number of relevant: 6 and median rating: 3.5
for userid: 563 number of relevant: 10 and median rating: 4.0
for userid: 623 number of relevant: 1 and median rating: 3.0
for userid: 14 number of relevant: 4 and median rating: 2.25
for userid: 72 number of relevant: 5 and median rating: 4.0
for userid: 451 number of relevant: 1 and median rating: 3.0
for userid: 467 number of relevant: 2 and median rating: 3.0
for userid: 379 number of relevant: 3 and median rating: 4.0
for userid: 310 number of relevant: 1 and median rating: 3.0
for userid: 29 number of relevant: 7 and median rating: 4.0


**Результаты apk, and mapk для подхода, реализованный выше по формуле.**
В ключе сам уникальный индекс человека, а справа 3 числа (apk5, apk10, apk100).

In [600]:
apkall

{14: [0.0, 0.0, 0.0],
 29: [0.0, 0.0, 0.0],
 72: [0.0, 0.0, 0.0],
 310: [0.0, 0.0, 0.0],
 379: [0.0, 0.0, 0.0],
 451: [0.0, 0.0, 0.0],
 467: [0.0, 0.0, 0.012195121951219513],
 546: [0.0, 0.0, 0.0023148148148148147],
 563: [0.0, 0.0, 0.002],
 623: [0.0, 0.0, 0.0]}

mapk: (mapk5, mapk10, mapk100).

In [601]:
mapk = np.array([apk for _, apk in apkall.items()]).sum(axis = 0)
mapk

array([0.        , 0.        , 0.01650994])

Данные метрики дают странные результаты. Значит по rmse, что я получил выше сложно что-то точное сказать. И сравнить с результами по метрикам apk, mapk.

Как можно улучшить работу модели?

<b>Уже много вариантов пытался сделать выше, в целом по поводу метрик mapk, apk не совсем понятно что они демонстрируют, так как данные очень разреженные и что назыввать релевантным и нерелевантным фильмом для конкретного человека тоже не ясно. А понимать это нужно, ибо метрики apk and mapk показывают насколько наше **ранжирование** - "хорошо", а вот это "хорошо" и не ясно что считать. Также меры сходства между пользователями ведут странно, если считать ближайших объектов на основе данных метрик сходства, то алгоритм может считать пары пользователей p, q - схожими, если у них всего 2-3 общих фильма, что достаточно странно!!! Я пытался ввести свою, взвешанную меру сходства с коэффициентом: |I(p) /\ I(q)| / |I(p) / I(q)| = M(p, q)(мера Жаккара) - который как-то бы учитывал тот факт, что у "похожих" пользователей хорошо, чтобы и было относительно большое пересечение в просмотренных фильмах то есть пытался комбинировать pearson_similarities(p, q) * M(p, q) or euclidean_similarities(p, q) * M(p, q). Что тоже не дало чего-то сильно отличенного от других способов, но хотя бы с данным весом мне knearest выдавал пары пользователей у которых было ощутимое пересечение в просмотренных фильмах! 

Далее, пытался воспользоваться одним из базовых подходов GroupLens algorithm - это то, что я сделал последним. RMSE был не такой и большой пор-ка 0.11 на тестовых данных (выкинул для каждого пользователя по 30 фильмов, случайно без внимания на rating!) хотя это тоже может вызвать вопрос: "А что считать хорошим RMSE?" 

Можно было бы попробовать множество различных SVD методов, но уже мало времени. Тут необходимо учитывать специфику данных, от этого очень многие методы несостоятельны! </b>